In [1]:
import torch as th


In [2]:
x = th.tensor([1,2,3,4,5])

In [3]:
x


tensor([1, 2, 3, 4, 5])

In [4]:
import syft as sy

/home/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
hook = sy.TorchHook(th)

In [6]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [7]:
bob = sy.VirtualWorker(hook,id="Bob")

In [8]:
x=x.send(bob)


In [9]:
bob._objects

{92548856757: tensor([1, 2, 3, 4, 5])}

In [10]:
x

(Wrapper)>[PointerTensor | me:73497806089 -> Bob:92548856757]

In [11]:
x.location

<VirtualWorker id:Bob #objects:1>

In [12]:
x.id_at_location

92548856757

In [13]:
x.id

73497806089

In [14]:
x.owner

<VirtualWorker id:me #objects:0>

In [15]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [16]:
x

(Wrapper)>[PointerTensor | me:73497806089 -> Bob:92548856757]

In [17]:
x=x.get()
x

tensor([1, 2, 3, 4, 5])

In [18]:
bob._objects

{}

In [19]:
alice = sy.VirtualWorker(hook,id="alice")

In [20]:
x=th.tensor([1,2,3,4,5])

In [21]:
x_ptr = x.send(bob,alice)

In [22]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [23]:
x=th.tensor([1,2,3,4,5]).send(bob,alice)

In [24]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [25]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad= True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [26]:
weights = th.tensor([[0.],[0.]],requires_grad= True).send(bob)

In [27]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad*=0
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


In [28]:
bob = bob.clear_objects()

In [29]:
bob._objects


{}

In [30]:
x=th.tensor([1,2,3,4,5]).send(bob)

In [31]:
bob._objects

{86989512523: tensor([1, 2, 3, 4, 5])}

In [32]:
del x


In [33]:
bob._objects

{}

In [34]:
from torch import nn,optim

In [35]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad= True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [36]:
model = nn.Linear(2,1)

In [37]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [38]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()
        pred=model(data)
        loss=((pred-target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
train()

tensor(0.5157)
tensor(0.3281)
tensor(0.2130)
tensor(0.1390)
tensor(0.0910)
tensor(0.0598)
tensor(0.0395)
tensor(0.0262)
tensor(0.0175)
tensor(0.0117)
tensor(0.0079)
tensor(0.0054)
tensor(0.0037)
tensor(0.0025)
tensor(0.0018)
tensor(0.0012)
tensor(0.0009)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)


In [39]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [40]:
data_alice = data[0:2].send(alice)
target_alice= target[0:2].send(alice)

In [41]:
datasets=[(data_bob,target_bob),(data_alice,target_alice)]

In [42]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            
            model = model.send(_data.location)
            
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            model = model.get()

            print(loss.get())

In [43]:
train()

tensor(1.6885, requires_grad=True)
tensor(0.0254, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0087, requires_grad=True)
tensor(0.0072, requires_grad=True)
tensor(0.0060, requires_grad=True)
tensor(0.0050, requires_grad=True)
tensor(0.0042, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0029, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(8.8123e-05, r

In [44]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [45]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [46]:
x=x.send(alice)

In [47]:
bob._objects

{40173842318: tensor([1, 2, 3, 4, 5])}

In [48]:
alice._objects

{27596507153: (Wrapper)>[PointerTensor | alice:27596507153 -> Bob:40173842318]}

In [49]:
y = x+x
y

(Wrapper)>[PointerTensor | me:17160990432 -> alice:57496093745]

In [50]:
bob._objects

{40173842318: tensor([1, 2, 3, 4, 5]),
 69116459549: tensor([ 2,  4,  6,  8, 10])}

In [51]:
alice._objects

{27596507153: (Wrapper)>[PointerTensor | alice:27596507153 -> Bob:40173842318],
 57496093745: (Wrapper)>[PointerTensor | alice:57496093745 -> Bob:69116459549]}

In [52]:
jon = sy.VirtualWorker(hook, id="jon")

In [53]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [54]:
bob._objects

{65856049783: tensor([1, 2, 3, 4, 5])}

In [55]:
alice._objects

{38329269408: (Wrapper)>[PointerTensor | alice:38329269408 -> Bob:65856049783]}

In [56]:
x=x.get()
x

(Wrapper)>[PointerTensor | me:38329269408 -> Bob:65856049783]

In [57]:
bob._objects

{65856049783: tensor([1, 2, 3, 4, 5])}

In [58]:
alice._objects

{}

In [59]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [60]:
bob._objects


{}

In [61]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [62]:
bob._objects

{48379272541: tensor([1, 2, 3, 4, 5])}

In [63]:
alice._objects

{36927521455: (Wrapper)>[PointerTensor | alice:36927521455 -> Bob:48379272541]}

In [64]:
del x

In [65]:
bob._objects

{}

In [66]:
alice._objects

{}

In [67]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [68]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [69]:
bob._objects

{88737353213: tensor([1, 2, 3, 4, 5])}

In [70]:
alice._objects

{}

In [71]:
x.move(alice)

(Wrapper)>[PointerTensor | me:30713357675 -> alice:30713357675]

In [72]:
bob._objects

{}

In [73]:
alice._objects

{30713357675: tensor([1, 2, 3, 4, 5])}

In [84]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [86]:
bob._objects

{39831138910: tensor([1, 2, 3, 4, 5]), 77108007435: tensor([1, 2, 3, 4, 5])}

In [87]:
alice._objects

{74313459392: (Wrapper)>[PointerTensor | alice:74313459392 -> Bob:77108007435]}

In [88]:
x.remote_get() #pull data from bob to alice

(Wrapper)>[PointerTensor | me:96046553980 -> alice:74313459392]

In [89]:
bob._objects

{39831138910: tensor([1, 2, 3, 4, 5])}

In [90]:
alice._objects

{74313459392: tensor([1, 2, 3, 4, 5])}

In [91]:
x.move(bob)

(Wrapper)>[PointerTensor | me:96046553980 -> Bob:96046553980]

In [92]:
x

(Wrapper)>[PointerTensor | me:96046553980 -> Bob:96046553980]

In [93]:
bob._objects

{39831138910: tensor([1, 2, 3, 4, 5]), 96046553980: tensor([1, 2, 3, 4, 5])}

In [94]:
alice._objects

{}